# Agenda

1. Magic methods
2. Context managers (how objects can behave in `with` blocks)
3. Static and class methods
4. Multiple inheritance and the MRO 
5. Python object model + hierarchy
6. Properties (looks like data, acts like methods)
7. Descriptors
8. Dataclasses

# Magic methods

We can define methods on our classes, and then access them via our instances. Generally speaking, we only want a method to be invoked when we do it explicitly.

But there are a whole slew of methods, which we call "dunder methods," or "magic methods," which we almost never invoke directly ourselves.  Rather, Python looks for these methods, and then invokes them under certain circumstances.

In [1]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
p = Person('name1')    
print(p.greet())

Hello, name1!


In [2]:
vars(p)  # this returns the dict of attributes set on the instance

{'name': 'name1'}

In [3]:
p.name  # retrieve the attribute named "name" from the object that p refers to


'name1'

In [4]:
# ICPO -- attribute lookup: instance, class, parent, object

print(p)   # this calls p.__str__() 

In [7]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    
    def __repr__(self):
        return f'Instance of Person, {vars(self)=}'    # as of Python 3.8, = after variable in f-string
    
p = Person('name1')    
print(p.greet())
print(p)

Hello, name1!
Instance of Person, vars(self)={'name': 'name1'}


In [8]:
# example: len(something)

# when we call len(something), the "len" function calls something.__len__() 

len(p)  # what's the length of our person?

TypeError: object of type 'Person' has no len()

In [10]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Instance of Person, {vars(self)=}'    # as of Python 3.8, = after variable in f-string
    
    def __len__(self):
        return len(self.name)  # how long is the name?
    
p = Person('name1')    
print(p.greet())
print(p)
print(len(p))  # I call len(p)... len calls p.__len__() 

Hello, name1!
Instance of Person, vars(self)={'name': 'name1'}
5


In [12]:
p[3]   # what happens when I do this?

TypeError: 'Person' object is not subscriptable

In [13]:
# when we say x[i], what's really being run behind the scenes is x.__getitem__(i)
# [] are "syntactic sugar," easier for us to write and read, but rewritten behind the scenes by the language

In [18]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Instance of Person, {vars(self)=}'    # as of Python 3.8, = after variable in f-string
    
    def __len__(self):
        return len(self.name)  # how long is the name?
    
    def __getitem__(self, index):
        print(f'\t{index=}')
        return self.name[index]
    
p = Person('name1')    
print(p[3])    # this is like saying p.__getitem__(3), which will return p.name[3]

	index=3
e


In [19]:
print(p[-100])

	index=-100


IndexError: string index out of range

In [20]:
# what if I ask for a slice?
print(p[1:4])   # slice from index 1 until (and not including) index 4

	index=slice(1, 4, None)
ame


In [21]:
slice(5)   # same as saying [:5]

slice(None, 5, None)

In [22]:
slice(1,4)

slice(1, 4, None)

In [23]:
slice(1,4,2)  # same as [1:4:2]

slice(1, 4, 2)

In [24]:
s = 'abcdefghijklmnopqrstuvwxyz'

s[10:20]   # from 10 until (not including) 20

'klmnopqrst'

In [25]:
# the above is rewritten to be:
s[slice(10, 20)]     # [10:20] is turned into this

'klmnopqrst'

In [26]:
s[10:20:3]   # same as s[slice(10, 20, 3)]

'knqt'

# What happens when we add objects together with `+`?

```python
3 + 5      # adding two integers, we get a new integer
'a' + 'bc' # adding two strings, we get a new string
```

Whenever we add two objects in Python, we're actually invoking a method. The `+` operator is translated into a call to the `__add__` method.

If I say `x + y` in Python, the language rewrites this to be `x.__add__(y)`.  In other words, we invoke the `__add__` method on whatever is on the *left* side. The second operand is then passed as an argument.

In [27]:
p1 = Person('name1')
p2 = Person('name2')

p1 + p2   # what will happen now?

TypeError: unsupported operand type(s) for +: 'Person' and 'Person'

In [28]:
x = 10
y = '20'

x + y   

TypeError: unsupported operand type(s) for +: 'int' and 'str'

In [29]:
# what happens if I add them, but in the opposite order?

y + x

TypeError: can only concatenate str (not "int") to str

In [36]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Person object, {vars(self)=}'
    
    def __add__(self, other):          # other is the object on the right side of the +
        if hasattr(other, 'name'):     # does the attribute 'name' exist on other?
            return Person(self.name + other.name)  # create a new person with the combined name
        else:
            return Person(self.name + str(other))
    
p1 = Person('name1')
p2 = Person('name2')

p1 + p2  # this invokes p1.__add__(p2) --> Person.__add__(p1, p2)

Person object, vars(self)={'name': 'name1name2'}

In [37]:
p1 + 'xxxxx'

Person object, vars(self)={'name': 'name1xxxxx'}

In [38]:
'yyyyy' + p1      # 'yyyyy'.__add__(p1)

TypeError: can only concatenate str (not "Person") to str

In [39]:
# If we try to invoke a method like __add__ and the method doesn't know what to do
# with an "other" argument of the current type, it will raise an exception, as 
# we've seen  *BUT FIRST* it will try to invoke the method in the opposite direction.

# Python does this via a value that it returns from the method called
# NotImplemented

NotImplemented

NotImplemented

In [40]:
type(NotImplemented)

NotImplementedType

In [41]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Person object, {vars(self)=}'
    
    def __add__(self, other):          # other is the object on the right side of the +
        if hasattr(other, 'name'):     # does the attribute 'name' exist on other?
            return Person(self.name + other.name)  # create a new person with the combined name
        else:
            return Person(self.name + str(other))
        
    def __radd__(self, other):   # this is invoked when Python tries +, swapping the order of the arguments
        return self.__add__(other)
    
p1 = Person('name1')
p2 = Person('name2')

p1 + p2  # this invokes p1.__add__(p2) --> Person.__add__(p1, p2)

Person object, vars(self)={'name': 'name1name2'}

In [42]:
p2 + p1

Person object, vars(self)={'name': 'name2name1'}

In [43]:
p1 + 'xxxx'

Person object, vars(self)={'name': 'name1xxxx'}

In [44]:
'yyy' + p1

Person object, vars(self)={'name': 'name1yyy'}

In [45]:
class Person:
    def __init__(self, name):
        self.name = name
        
    def greet(self):
        return f'Hello, {self.name}!'
    
    def __repr__(self):
        return f'Person object, {vars(self)=}'
    
    def __add__(self, other):          # other is the object on the right side of the +
        if hasattr(other, 'name'):     # does the attribute 'name' exist on other?
            return Person(self.name + other.name)  # create a new person with the combined name
        else:
            return Person(self.name + str(other))
        
    def __radd__(self, other):   # this is invoked when Python tries +, swapping the order of the arguments
        if hasattr(other, 'name'):     # does the attribute 'name' exist on other?
            return Person(other.name + self.name)
        else:
            return Person(str(other) + self.name)

    
p1 = Person('name1')
p2 = Person('name2')

p1 + p2  # this invokes p1.__add__(p2) --> Person.__add__(p1, p2)

Person object, vars(self)={'name': 'name1name2'}

In [46]:
p1 + 'xxxxx'

Person object, vars(self)={'name': 'name1xxxxx'}

In [47]:
'yyyyy' + p2

Person object, vars(self)={'name': 'yyyyyname2'}

# Exercise: SortedList

1. Define a new class, `SortedList`, that we'll assume will only contain integers.
2. When you create a new instance of `SortedList`, you pass a list of integers.  These are then stored internally, in sorted order.  (It's totally OK to use `sorted` in this exercise.)
3. Have `__repr__` return a string with the sorted items.
4. Have `__len__` return the number of items in our list.
5. If you invoke `+` on a `SortedList` and another `SortedList`, the result is a new `SortedList` instance with all of the original elements, in sorted order.
6. If you invoke `+` on something else (should be an integer), then that should be added, in its appropriate place, to the `SortedList` instance.

Example:

    sl1 = SortedList([10, 5, 18])
    sl2 = SortedList([9,7,12])
    print(sl1)  # [5, 10, 18]
    print(sl2)  # [7, 9, 12]

    sl3 = sl1 + 7 
    print(sl3)  # [5, 7, 10, 18]
    
    sl4 = 8 + sl1
    print(sl4)  # [5, 8, 10, 18]
    
    print(sl1 + sl2)   # [5, 7, 9, 10, 12, 18]